# FLEKS Python Visualization Toolkit: Test Particle Data

flekspy is a Python package for processing FLEKS data. This notebook focuses on handling test particle data.

## FLEKS data format

* Field: *.out format or AMREX built-in format, whose directory name is assumed to end with "_amrex"
* PIC particle: AMREX built-in format 
* Test particle: binary data format

## Downloading demo data

Example test particle data can be downloaded as follows:

In [ ]:
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/test_particles_PBEG.tar.gz"
download_testfile(url, "data")

## Plotting Test Particle Trajectories

### Loading particle data

In [ ]:
from flekspy import FLEKSTP

tp = FLEKSTP("data/test_particles_PBEG", iSpecies=1)

### Obtaining particle IDs

Test particle IDs consists of a CPU index and a particle index attached to the CPU.

In [ ]:
tp.getIDs()

### Reading particle trajectory

In [ ]:
tp[0].collect()

### Getting initial location

In [ ]:
x = tp.read_initial_condition(tp.getIDs()[0])
print("time, X, Y, Z, Vx, Vy, Vz")
print(
    f"{x[0]:.2e}, {x[1]:.2e}, {x[2]:.2e}, {x[3]:.2e}, {x[4]:.2e}, {x[5]:.2e}, {x[6]:.2e}"
)

### Interpolate particles trajectory at selected times

In [ ]:
from flekspy.tp import interpolate_at_times

interpolate_at_times(tp[0], [1.0])

### Plotting trajectory

In [ ]:
import matplotlib.pyplot as plt
tp.plot_trajectory(tp.getIDs()[0])
plt.show()

### Reading and visualizing all particle's location at a given snapshot

In [ ]:
ids, pData = tp.read_particles_at_time(0.0, doSave=False)
tp.plot_location(pData)
plt.show()

### Selecting particles starting in a region

In [ ]:
from flekspy.tp import Indices


def f_select(tp, pid):
    pData = tp.read_initial_condition(pid)
    inRegion = pData[Indices.X] > 0 and pData[Indices.Y] > 0
    return inRegion


pSelected = tp.select_particles(f_select)

### Saving trajectories

Saving trajectories to CSV

In [ ]:
tp.save_trajectory(tp.getIDs()[0], format="csv")

Saving trajectories to Parquet

In [ ]:
tp.save_trajectory(tp.getIDs()[0], format="parquet")

Saving trajectories to HDF5

In [ ]:
tp.save_trajectories([0, 1], filename="my_tp.h5")

Writing large amount of data to a single HDF5 file is slow. For parallel writing to a series of HDF5 files in chunks, see the sample job script together with the conversion script.

### Calculating drifts

With the $\mathbf{E}, \mathbf{B}$, and $\nabla \mathbf{B}$ saved along the trajectory, we can compute various drifts:
- Convection drift velocity: `tp.get_ExB_drift(pid)`
- Curvature drift velocity: `tp.get_curvature_drift(pid)`
- Gradient drift velocity: `tp.get_gradient_drift(pid)`
- Polarization drift velocity: `tp.get_polarization_drift(pid)`

The first adiabatic assumption can be checked by comparing the ratio between the curvature length and the gyroradius $r_c / r_L$ via `get_adiabaticity_parameter`. If $r_c / r_L \gg 1$, then the gyromotion can be savely ignored.


The Betatron acceleration $\mu \partial B/\partial t$ can be indirectly computed with the knowledge of total derivative $\mathrm{d}B/\mathrm{d}t$ and $\mathbf{v}\nabla B$ along the trajectory:

In [ ]:
pid = tp.getIDs()[0]
pt = tp[pid]
mu = tp.get_first_adiabatic_invariant(pt)
pt = tp.get_betatron_acceleration(pt, mu)

To analyze a specific drift:

In [ ]:
tp.analyze_drift(pid, "gradient")

By combining all these together, we come up with time series of various terms:

In [ ]:
tp.analyze_drifts(pid)

We can the track the energy change from drift and EM field variations:

In [ ]:
from flekspy.tp import plot_integrated_energy

df = tp.integrate_drift_accelerations(tp.getIDs()[0])
plot_integrated_energy(df)

Alternatively, the energy change can be decomposed into three terms: the parallel acceleration, Betatron acceleration, and the Fermi acceleration:

In [ ]:
df = tp.get_energy_change_guiding_center(tp.getIDs()[0])

df